In [1]:
from builder import Builder, Process

In [2]:
b = Builder()

In [3]:
b.list_processes()

['console-emitter', 'ram-emitter']


In [4]:
from process_bigraph.experiments.minimal_gillespie import GillespieEvent , GillespieInterval
b.register_process('GillespieEvent', GillespieEvent)
b.register_process('GillespieInterval', GillespieInterval)

In [5]:
b.list_processes()

['GillespieEvent', 'console-emitter', 'ram-emitter', 'GillespieInterval']


In [6]:
## add processes
b['event_process'].add_process(
    name='GillespieEvent',
    kdeg=1.0,  # kwargs fill parameters in the config
)

In [7]:
# b.compile()

In [8]:
b['event_process'].ports(True)

{
'_inputs': {
   'mRNA': 'map[float]',
   'DNA': {
      'A gene': 'float',
      'B gene': 'float'
  }
},
'_outputs': {
   'mRNA': 'map[float]'
}
}


In [9]:
# b.schema

In [10]:
b['event_process'].schema

{'_type': 'process',
 '_default': {'inputs': {}, 'outputs': {}},
 '_apply': 'process_bigraph.composite.apply_process',
 '_serialize': 'process_bigraph.composite.serialize_process',
 '_deserialize': 'process_bigraph.composite.deserialize_process',
 '_check': 'process_bigraph.composite.check_process',
 '_type_parameters': ['inputs', 'outputs'],
 '_description': '',
 'inputs': {'_type': 'wires',
  '_default': {},
  '_apply': 'bigraph_schema.registry.apply_tree',
  '_serialize': 'bigraph_schema.type_system.serialize_tree',
  '_deserialize': 'bigraph_schema.type_system.deserialize_tree',
  '_divide': 'bigraph_schema.type_system.divide_tree',
  '_check': 'bigraph_schema.type_system.check_tree',
  '_type_parameters': ['leaf'],
  '_description': 'mapping from str to some type in a potentially nested form',
  '_leaf': {'_type': 'path',
   '_default': [],
   '_apply': 'bigraph_schema.type_system.apply_path',
   '_check': 'bigraph_schema.type_system.check_list',
   '_serialize': 'bigraph_schema.t

In [11]:
b['event_process'].connect(port='mRNA', target=['mRNA_store'])
b

Exception: cannot wire mRNA as we are already at the top level {'mRNA': 'map[float]', 'DNA': {'A gene': 'float', 'B gene': 'float'}}

In [ ]:
b.visualize()

In [ ]:
# from builder.builder_api import Builder, dict_from_builder_tree
# tree_dict = dict_from_builder_tree(b.tree)

# tree_dict

In [ ]:
@b.register_process('toy')
class Toy(Process):
    config_schema = {
        'A': 'float',
        'B': 'float',
    }

    def __init__(self, config, core):
        super().__init__(config, core)

    def schema(self):
        return {
            'inputs': {
                'A': 'float',
                'B': 'float'},
            'outputs': {
                'C': 'float'}
        }

    def update(self, state, interval):
        update = {
            'C': state['A'] + state['B']
        }
        return update


In [ ]:
b.list_processes()

In [ ]:
b['toy'].add_process(name='toy')

In [ ]:
b['toy'].ports()

In [ ]:
b.visualize()

In [ ]:
b['toy'].connect(port='A', target='A_store')
# b['toy'].connect(port='B', target='B_store')  # TODO -- compile is trying to connect A at the wrong level??
# b['toy'].connect(port='C', target='C_store')

In [ ]:
b.plot()

In [ ]:
b.document()

In [ ]:
b.write(filename='toy_bigraph')